# Big Data in Finance: Part II <BR><BR> Homework 1: CRSP and Compustat
    
**Instructor** <BR>
Kirsten Burr, kburr26@gsb.columbia.edu
    
**TA's:** <BR>
Meha Sadasivam, MSadasivam21@gsb.columbia.edu <BR>
Daheng Yang, dyang22@gsb.columbia.edu

**Due:** Feb 15, 2024, by 11:59 pm. Late submission will not be accepted.

**Goal**: Explore the information CRSP and Compustat

# Description

US stock exchanges finished 2023 with their best performances in years. The S&P 500-stock index posted its ninth consecutive weekly gain -- its longest winning streak since 2004 -- as it jumped more than 24 percent for the year. The blue-chip Dow Jones industrial average gained 13.7 percent for the year, and the tech-heavy Nasdaq soared 43.4 percent -- its highest annual gain since 2020.

Let's look at what happened with stock returns using the data downloaded from CRSP and Compustat.

## Part I: CRSP stock returns
1. What was the cumulative average returns (market-cap weighted and equal-weighted) of CRSP stocks in 2023? (i.e. how much would an investor have by the end of 2023 had she invested $1 in CRSP stocks at the beginning of 2023?)
2. Plot the market-cap weighted and equal-weighted cumulative average returns of CRSP stocks from the beginning of the year. 
3. How much would the same investor have had she entered the market at the end of June, during which stock volatility was relatively lower? 
4. What were the 10 top performing stocks in 2023? 

Note: You should only consider ordinary common stocks (i.e., shrcd equal to 10 or 11) that are listed in one of the three main stock exchanges (i.e., exchcd equal to 1 (NYSE), 2 (NYSE MKT) or 3 (NASDAQ)). Remember to use the prior month's market-cap when market-weighting returns.

## Part II: Tech stock returns
1. How much would an investor have by the end of 2023 had she invested $1 in a TECH portfolio (market-cap weighted and equal-weighted) at the beginning of 2023? 
2. Plot the market-cap weighted and equal-weighted cumulative average returns of the Tech portfolio from the beginning of the year. 
3. How much would the same investor have had she entered the market at the end of June? 
4. What are the 10 top performing Tech stocks in the year of 2023? 

Note: The closest definition of Tech firms according to Fama and French is Business Equipment industry, which includes Computers, Software, and Electronic Equipment. These are firms with SIC codes in the following ranges:

        3570-3579
        3660-3692
        3694-3699
        3810-3829
        7370-7379

## Part III: High profitability stock returns
1. Given the fundamental data available to investors as of December 31, 2022, what are the 10 most profitable companies in 2022? Use operating profitability normalized by book-equity (OPBE) from the lecture notes as a measure of profitability.
2. How much an investor would have by the end of 2023 had she invested instead in a portfolio (market-cap weighted and equal-weighted) of these 10 most profitable firms' stocks at the beginning of 2023? 
3. Compare your results to those from Part II. Which portfolio performed better? Why do you think this is the case?
4. Plot the market-cap weighted and equal-weighted cumulative average returns from the beginning of the year.
5. How much would the same investor have had she invested in these same 10 firms at the end of June? 

Note: Publicly-listed companies must report their financials within 91 days after the end of the fiscal period. Most companies report on the deadline. Consider this when merging financial results to stock performance.

# Code 

## Packages

Below is some code that will help you get started. Make sure you have installed all required packages. Use "conda install" or "pip install" if you you are missing any of the packages.

In [ ]:
# Packages
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import wrds
from datetime import datetime, timedelta
from pandas.tseries.offsets import MonthEnd

# Setups
pd.set_option("display.max_rows", 100)

## WRDS Data Download

### WRDS Connection
Make sure you can connect to WRDS server. You should configure wrds connector before hand. Please check the syllabus for details.


In [ ]:
# Set Up WRDS connection
db = wrds.Connection(wrds_username='username_here') # make sure to change the username. 

### Download MSF

In [ ]:
sfvars = ['prc','vol','ret', 'retx','cfacpr','cfacshr']
query = """SELECT date, permno, permco, {} 
           FROM crsp.msf
           WHERE date>='2022-01-01'""".format(', '.join(sfvars))
msf_data = db.raw_sql(query, date_cols=['date'])

# check PK
assert msf_data.duplicated(subset=['date','permno']).sum() == 0

### Download MSEALL

In [ ]:
msevars = ['comnam', 'dlret', 'dlretx', 'exchcd','naics', 'cusip', 'ncusip', 'shrcd', 'shrout', 
           'siccd', 'ticker']
query = """SELECT date, permno, permco, {} 
           FROM crspq.mseall
           WHERE date>='2022-01-01'""".format(', '.join(msevars))
mse_data = db.raw_sql(query, date_cols=['date'])

# drop duplicates
mse_data = mse_data[~mse_data.duplicated()].copy()

# check PK
assert mse_data.duplicated(subset=['date','permno']).sum() == 0

### Merge MSF and MSEALL

In [ ]:
# sort
msf_data.sort_values(['date', 'permno'], inplace=True)
mse_data.sort_values(['date', 'permno'], inplace=True)

# Check of coverage
x = pd.Series(msf_data.permno.unique())
y = pd.Series(mse_data.permno.unique())
# assert len(x[~x.isin(y)]) == 0

# Check if delisting happens only once per security
assert mse_data[mse_data.dlret.notnull()].groupby(['permno']).date.value_counts().max() == 1

# Check if delisting returns only appears after the firm is not longer covered by msf
test = mse_data[mse_data.dlret.notnull()]
test = pd.merge(msf_data, test[['permno', 'dlret', 'date']], on=['permno'])
assert len(test[test.date_x>test.date_y]) == 0

In [ ]:
msf_data.permno = msf_data.permno.astype('int64')
msf_data.permco = msf_data.permco.astype('int64')

mse_data.permno = mse_data.permno.astype('int64')
mse_data.permco = mse_data.permco.astype('int64')

crsp = pd.merge_asof(msf_data, 
                     mse_data, 
                     on='date', 
                     by=['permno','permco'], 
                     direction='backward')

crsp.date = crsp.date + MonthEnd(0)

# Check Primary Key
assert crsp.duplicated(subset=['permno','date']).sum() == 0

In [ ]:
crsp.head()

## Preparing the Data

### Adjust returns for stock delisting

### Calculate market value of equity (ME) and one-month lagged ME

### Apply Share Type and Exchange Code Filters

## Part I -- CRSP Stock Returns

### Calculate CRSP value weighted returns

### Calculate CRSP equal weighted returns

### Plot cumulative average returns

### Report cumulative average returns

### Calculate cumulative returns

### Top 10 Stocks -- CRSP

## Part II -- Tech Stock Returns

### Calculate TECH value weighted returns

### Calculate TECH equal weighted returns

### Plot cumulative portfolio returns

### Report cumulative portfolio returns

### Top 10 Stocks -- TECH

## Part III -- OPBE Sorts

### Merge CRSP and Compustat to Calculate OPBE

#### Download Compustat data from 2022

In [ ]:
fund_table = 'funda'

varlist = ['conm', 'tic', 'cusip','fyear', 'fyr', 'at','capx', 'ceq', 'cogs', 'csho', 'dlc', 'dlcch','dltt', 'dp', 'ib', 'itcb', 
           'lt', 'mib', 'naicsh', 'ni', 'prstkcc', 'pstk', 'pstkl', 'pstkrv', 're', 'revt', 'sale', 
           'seq', 'sich', 'txdb', 'txdi', 'txditc', 'wcapch', 'xint', 'xlr', 'xrd', 'xsga']


query = """SELECT gvkey, datadate, {}
           FROM comp.{}
           WHERE datafmt = 'STD'
           AND popsrc = 'D'
           AND indfmt = 'INDL'
           AND consol = 'C'
           AND fyear>=2021;""".format(", ".join(varlist), fund_table)

compa = db.raw_sql(query, date_cols=['datadate'])
# Reporting date
compa['report_date'] = compa['datadate'] + timedelta(days=91)

#### Keep the latest (datadate) obervation if there are duplicates

#### Calculate OPBE

#### Merge CRSP and Compustat data

In [ ]:
sql = '''SELECT gvkey, lpermno as permno, lpermco as permco, linktype, linkdt, linkenddt
         FROM crsp.ccmxpf_lnkhist
         WHERE linktype IN ('LC', 'LU', 'LS') 
      ''' 

link_table = db.raw_sql(sql, date_cols=['linkdt', 'linkenddt'])

### Top 10 most profitable stocks of 2022

### Performance of top 10 most profitable stocks

#### Calculate market-cap weighted returns

#### Calculate equal weighted returns

### Report cumulative portfolio returns

### Compare results

[Insert answer here]

### Plot cumulative portfolio returns